In [1]:
import sys
import os
module_path = os.path.abspath('/media/jary/DATA/Uni/tesi/codice')

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from networks import NoKafnet, Kafnet
import utils.datasetsUtils.MINST as MINST
from utils.datasetsUtils.taskManager import SingleTargetClassificationTask, NoTask
import configs.configClasses as configClasses
from torchvision.transforms import transforms
import torch
import networks.continual_learning as continual_learning
import networks.continual_learning_beta as continual_learning_beta

from Trainer import Trainer
import matplotlib.pyplot as plt
from collections import defaultdict
from copy import deepcopy
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [3]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
GeForce GTX 1050


In [5]:
configOnline = configClasses.OnlineLearningConfig()
configOnline.L1_REG = 0
configOnline.EPOCHS = 20
configOnline.SAVE_PATH = './models/tecnique_comparison_multikaf'
configOnline.IS_CONVOLUTIONAL = False
configOnline.MODEL_NAME = ''
print(configOnline)

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 20
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
EWC_TYPE: <class 'networks.continual_learning.OnlineEWC'>
GAMMA: 1.0
IS_CONVOLUTIONAL: False
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: 
OPTIMIZER: SGD
RUN_NAME: default
SAVE_PATH: ./models/tecnique_comparison_multikaf
USE_EWC: True
USE_TENSORBOARD: True



In [6]:
dataset = MINST.PermutedMINST('../data/minst', download=True, n_permutation=4,
                        force_download=False, train_split=0.8, transform=None, target_transform=None)
dataset.load_dataset()

../data/minst/download
task #0 with train 56000 and test 14000 images (label: 0)
task #1 with train 56000 and test 14000 images (label: 1)
task #2 with train 56000 and test 14000 images (label: 2)
task #3 with train 56000 and test 14000 images (label: 3)


In [7]:
net = multikaf = Kafnet.MultiKAFMLP(len(dataset.class_to_idx), hidden_size=int(400*0.7),  kaf_init_fcn=None,
                             trainable_dict=False, kernel_combination='sum', kernels=['gaussian', 'sigmoid', 'softplus'])

In [8]:
results = []
experiments = [('no_cont_learning', None), 
               ('ewc', continual_learning.OnlineEWC),
#                ('gem', continual_learning.GEM),
               ('embedding', continual_learning_beta.embedding)]

In [9]:
for name, tec in experiments:
    print(name)
    
    n = deepcopy(net)
    config = deepcopy(configOnline)
    
    config.MODEL_NAME = name

    if 'name' == 'gem':
        config.EWC_IMPORTANCE = 0.5
        
    config.EWC_TYPE = tec
    
    trainer = Trainer(n, deepcopy(dataset), config, save_modality=2)

    r = trainer.load()
    if not r:
        r = trainer.all_tasks()
    
    results.append((name, r))
    
    del trainer
    del n

no_cont_learning


/media/jary/DATA/Uni/tesi/codice/Trainer.py:36: UserWarning: Ewc type is set to None  
  warnings.warn("Ewc type is set to None  ")
Training task 0, epoch 1: 100%|██████████| 875/875 [00:17<00:00, 50.66it/s, loss=2.13, batch#=875]
Testing task 0: 219it [00:03, 71.38it/s, batch#=219]
Training task 0, epoch 2: 100%|██████████| 875/875 [00:16<00:00, 52.67it/s, loss=1.45, batch#=875]
Testing task 0: 219it [00:03, 70.39it/s, batch#=219]
Training task 0, epoch 3: 100%|██████████| 875/875 [00:17<00:00, 50.50it/s, loss=0.873, batch#=875]
Testing task 0: 219it [00:03, 68.90it/s, batch#=219]
Training task 0, epoch 4: 100%|██████████| 875/875 [00:16<00:00, 51.95it/s, loss=0.62, batch#=875] 
Testing task 0: 219it [00:03, 69.51it/s, batch#=219]
Training task 0, epoch 5: 100%|██████████| 875/875 [00:16<00:00, 51.65it/s, loss=0.493, batch#=875]
Testing task 0: 219it [00:03, 69.16it/s, batch#=219]
Training task 0, epoch 6: 100%|██████████| 875/875 [00:16<00:00, 50.15it/s, loss=0.426, batch#=875]
Testi

Training task 2, epoch 3: 100%|██████████| 875/875 [00:18<00:00, 46.05it/s, loss=0.302, batch#=875]
Testing task 2: 219it [00:03, 60.56it/s, batch#=219]
Testing task 0: 219it [00:03, 61.37it/s, batch#=219]
Testing task 1: 219it [00:03, 61.61it/s, batch#=219]
Training task 2, epoch 4: 100%|██████████| 875/875 [00:18<00:00, 45.69it/s, loss=0.276, batch#=875]
Testing task 2: 219it [00:03, 60.79it/s, batch#=219]
Testing task 0: 219it [00:03, 61.38it/s, batch#=219]
Testing task 1: 219it [00:03, 61.27it/s, batch#=219]
Training task 2, epoch 5: 100%|██████████| 875/875 [00:18<00:00, 46.16it/s, loss=0.255, batch#=875]
Testing task 2: 219it [00:03, 60.36it/s, batch#=219]
Testing task 0: 219it [00:03, 61.10it/s, batch#=219]
Testing task 1: 219it [00:03, 60.59it/s, batch#=219]
Training task 2, epoch 6: 100%|██████████| 875/875 [00:18<00:00, 46.41it/s, loss=0.239, batch#=875]
Testing task 2: 219it [00:03, 61.51it/s, batch#=219]
Testing task 0: 219it [00:03, 62.02it/s, batch#=219]
Testing task 1: 2

KeyboardInterrupt: 

In [ ]:
print('RESULTS')
for name, r in results:
    print(name, r['metrics'])
    print()

In [ ]:
n_task = len(results[0][1]['tasks'])

tot_epochs = 0

for _, r in results:
    for k, v in r['tasks'].items():
        tot_epochs = max(tot_epochs, len(v['accuracy']))
      
fig, ax = plt.subplots(nrows=n_task, ncols=1, figsize=(22, 24), sharex=True, sharey=True)

for name, r in results:
    for i, task in enumerate(r['tasks'].keys()):

        com = r['tasks'][task]
        #no_ewt = metrics_no_ewt['tasks'][task]

        x = range(tot_epochs-len(com['accuracy']), tot_epochs)

        #ax = fig.add_subplot(n_task, 1, i+1, sharex=ax) 

        ax[i].plot(x, com['accuracy'], label=name)
        #ax.plot(x, no_ewt['accuracy'], label='online ewt')

        ax[i].set_xticks(range(0, tot_epochs, 5),minor=False)

        ax[i].set_title("Task {}".format(task))
        ax[i].legend(loc="lower left")
        ax[i].grid(True, axis='both')

        
fig.subplots_adjust(hspace=0.1, wspace=0.01)


In [ ]:
for name, tec in experiments:

    current_w = {n:p.cpu() for n, p in net.named_parameters() if p.requires_grad}

    x = np.arange(4)
    y = np.arange(len(current_w))
    
    xx, yy = np.meshgrid(x, y)
    z = np.zeros(shape=(len(x), len(y)))
    
    fig = plt.figure(figsize=(22, 22))
    ax = fig.add_subplot(111, projection='3d')
    
    x_labels = []
    
    for i in x:

        if i == 0:
            x_labels.append('0')
            
        else:
            x_labels.append('{}->{}'.format(i-1, i))
            
        n = deepcopy(net)
        config = deepcopy(configOnline)

        config.MODEL_NAME = name

        if 'name' == 'gem':
            config.EWC_IMPORTANCE = 0.5

        config.EWC_TYPE = tec

        trainer = Trainer(n, deepcopy(dataset), config, save_modality=2)

        r = trainer.load(i)
        
        diff = {n: torch.dist(p.cpu(), current_w[n]) for n, p in trainer.model.named_parameters() if p.requires_grad }
        current_w = {n: p.cpu() for n, p in trainer.model.named_parameters() if p.requires_grad }
        
        for j, (_, d) in enumerate(diff.items()):
            z[i, j] = d
    

    z_n = z.flatten()

    ax.view_init(45, 45)
    
    ax.bar3d(xx.flatten(),
          yy.flatten(),
          np.zeros(len(z_n)),
          0.2, 0.5, z_n )
    
    ax.set_xticks(x)
    ax.set_xticklabels(x_labels)
    
    ax.set_yticks(y)
    ax.set_yticklabels(list(diff.keys()), fontdict={'fontsize':12})
    
    ax.text(3.5, 0, 2, name.upper(), color='black',  fontsize='xx-large', 
        bbox=dict(facecolor='white', edgecolor='black'))
    
    plt.show()

In [ ]:
tot_epochs = 0

fig, ax = plt.subplots(nrows=len(experiments), ncols=4, figsize=(35, 40), 
                       sharex=True, sharey=True,  subplot_kw={'projection':'3d'})

for k in range(4):
    dataset.task = k
    
    images = []
    labels = []
    
    for im in range(300):
        d = dataset[im]
        if d[1] in [0, 3, 8]: 
            images.append(d[0])
            labels.append(d[1].numpy()[0])
#     colors = [int(i % 23) for i in labels]

    for i, (name, tec) in enumerate(experiments):

        tsne_model_en_2d = PCA(n_components=3, random_state=19)

        n = deepcopy(net)
        config = deepcopy(configOnline)

        config.MODEL_NAME = name

        if 'name' == 'gem':
            config.EWC_IMPORTANCE = 0.5

        config.EWC_TYPE = tec

        trainer = Trainer(n, deepcopy(dataset), config, save_modality=2)

        r = trainer.load()

        with torch.no_grad():
            a = trainer.model.embedding(torch.stack(images, 0).to(config.DEVICE)).cpu().numpy()
            b = tsne_model_en_2d.fit_transform(a)
            
#             for l in set(labels):
#                 ix = np.where(labels == l)
            ax[i][k].scatter(b[:, 0], b[:, 1], b[:, 2], c=labels)
            
        del trainer
        del n
plt.show()